In [ ]:
import logging
import sys
import pathlib
from glob import glob
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
import tifffile
import matplotlib.pyplot as plt

sys.path.append('../src')
import settings
import annotation_utils

In [ ]:
# Raw training and test data
df_hubmap_hpa_organ_segmentation_metadata = pd.concat((
    pd.read_csv(settings.DATA / 'train_metadata.csv'),
    pd.read_csv(settings.DATA / 'test_metadata.csv')
), axis=0, ignore_index=True)

# HPA dictionary data
df_hpa_metadata = pd.read_csv(settings.DATA / 'external_data' / 'HPA' / 'metadata.csv')
df_hpa_metadata = df_hpa_metadata.loc[df_hpa_metadata['rle'].notna(), :]
# HuBMAP Colonic Crypt data
df_hubmap_colonic_crypt_metadata = pd.read_csv(settings.DATA / 'external_data' / 'HuBMAP_Colonic_Crypt' / 'metadata.csv')

In [ ]:
def visualize_annotations(image, rle_mask, transpose_mask, metadata, path=None):

    """
    Visualize image along with its annotations

    Parameters
    ----------
    image (path-like str or numpy.ndarray of shape (height, width, 3)): Image path relative to root/data or image array
    rle_mask (str): Run-length encoded segmentation mask string
    transpose_mask (bool): Whether to transpose mask or not
    metadata (dict): Dictionary of metadata used in the visualization title
    path (path-like str or None): Path of the output file or None (if path is None, plot is displayed with selected backend)
    """

    if isinstance(image, pathlib.Path) or isinstance(image, str):
        # Read image from the given path
        image_path = image
        print(image_path)
        if image_path.split('.')[-1] == 'tif' or image_path.split('.')[-1] == 'tiff':
            image = tifffile.imread(str(settings.DATA / image_path))
        else:
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    elif isinstance(image, np.ndarray):
        title = ''
    else:
        # Raise TypeError if image argument is not an array-like object or a path-like string
        raise TypeError('Image is not an array or path.')

    if rle_mask is not None:
        mask = annotation_utils.decode_rle_mask(rle_mask=rle_mask, shape=image.shape[:2])
        if transpose_mask:
            mask = mask.T

    if rle_mask is not None:
        fig, axes = plt.subplots(figsize=(48, 20), ncols=3)
    else:
        fig, axes = plt.subplots(figsize=(16, 20), ncols=1)

    if rle_mask is not None:
        axes[0].imshow(image)
        axes[1].imshow(image)
    else:
        axes.imshow(image)

    if rle_mask is not None:
        axes[1].imshow(mask, alpha=0.5)
        axes[2].imshow(mask)
    
    if rle_mask is not None:
        for i in range(3):
            axes[i].set_xlabel('')
            axes[i].set_ylabel('')
            axes[i].tick_params(axis='x', labelsize=15, pad=10)
            axes[i].tick_params(axis='y', labelsize=15, pad=10)
            
        axes[0].set_title('Image', size=20, pad=15)
        axes[1].set_title('Image + Mask', size=20, pad=15)
        axes[2].set_title('Mask', size=20, pad=15)
        
        fig.suptitle(
            f'''
            Image ID {metadata["id"]} - {metadata["organ"]} - {metadata["data_source"]} - {metadata["age"]} - {metadata["sex"]}
            Image Shape: {metadata["image_height"]}x{metadata["image_width"]} - Pixel Size: {metadata["pixel_size"]}µm - Tissue Thickness: {metadata["tissue_thickness"]}µm
            ''',
            fontsize=25
        )
    else:
        axes.set_xlabel('')
        axes.set_ylabel('')
        axes.tick_params(axis='x', labelsize=15, pad=10)
        axes.tick_params(axis='y', labelsize=15, pad=10)
        axes.set_title(
            f'''
            Image ID {metadata["id"]} - {metadata["organ"]} - {metadata["data_source"]} - {metadata["age"]} - {metadata["sex"]}
            Image Shape: {metadata["image_height"]}x{metadata["image_width"]} - Pixel Size: {metadata["pixel_size"]}µm - Tissue Thickness: {metadata["tissue_thickness"]}µm
            ''',
            size=20,
            pad=15
        )

    if path is None:
        plt.show()
    else:
        plt.savefig(path)
        plt.close(fig)


In [ ]:
for idx, row in df_hubmap_hpa_organ_segmentation_metadata.iloc[350:].iterrows():

    image = cv2.imread(row['image_filename'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rle_mask = row['rle']

    visualize_annotations(
        image=image,
        rle_mask=rle_mask,
        transpose_mask=True,
        metadata=row.to_dict(),
    )

In [ ]:
for idx, row in df_hpa_metadata.iterrows():

    image = cv2.imread(row['image_filename'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rle_mask = row['rle']

    visualize_annotations(
        image=image,
        rle_mask=rle_mask,
        transpose_mask=False,
        metadata=row.to_dict(),
    )

In [ ]:
for idx, row in df_hubmap_colonic_crypt_metadata.iterrows():

    image = cv2.imread(row['image_filename'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    rle_mask = row['rle']

    visualize_annotations(
        image=image,
        rle_mask=rle_mask,
        transpose_mask=False,
        metadata=row.to_dict(),
    )